# 如何为可运行对象添加回退机制
在使用语言模型时，您可能会经常遇到底层API的问题，无论是速率限制还是停机。因此，当您将LLM应用程序投入生产时，防范这些问题变得越来越重要。这就是我们引入回退概念的原因。
**后备方案**（fallback）是指在紧急情况下可以采用的替代计划。
关键的是，回退机制不仅可以应用在LLM层面，还能在整个可运行层面实施。这一点非常重要，因为不同模型通常需要不同的提示词。因此，当调用OpenAI失败时，你并不想直接将相同的提示词发送给Anthropic——你可能需要使用不同的提示模板，并发送一个调整后的版本。

## 大语言模型API错误的回退方案
这可能是后备方案最常见的用例之一。向大型语言模型(LLM)API发出的请求可能因多种原因失败——API可能宕机、可能触及速率限制，或是其他各种问题。因此，使用后备方案有助于防范此类情况。
重要提示：默认情况下，许多LLM封装器会捕获错误并自动重试。在使用后备方案时，您很可能需要关闭这些功能。否则，第一个封装器会持续重试而不会报错。

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai

In [1]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

首先，模拟一下如果我们遇到OpenAI的RateLimitError会发生什么

In [2]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")

In [3]:
# Note that we set max_retries = 0 to avoid retrying on RateLimits, etc
openai_llm = ChatOpenAI(model="gpt-4o-mini", max_retries=0)
anthropic_llm = ChatAnthropic(model="claude-3-haiku-20240307")
llm = openai_llm.with_fallbacks([anthropic_llm])

In [4]:
# Let's use just the OpenAI LLm first, to show that we run into an error
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

Hit error


In [28]:
# Now let's try with fallbacks to Anthropic
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

content=' I don\'t actually know why the chicken crossed the road, but here are some possible humorous answers:\n\n- To get to the other side!\n\n- It was too chicken to just stand there. \n\n- It wanted a change of scenery.\n\n- It wanted to show the possum it could be done.\n\n- It was on its way to a poultry farmers\' convention.\n\nThe joke plays on the double meaning of "the other side" - literally crossing the road to the other side, or the "other side" meaning the afterlife. So it\'s an anti-joke, with a silly or unexpected pun as the answer.' additional_kwargs={} example=False


我们可以像使用普通大语言模型一样使用我们的“带后备方案的大语言模型”。

In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError:
        print("Hit error")

content=" I don't actually know why the kangaroo crossed the road, but I can take a guess! Here are some possible reasons:\n\n- To get to the other side (the classic joke answer!)\n\n- It was trying to find some food or water \n\n- It was trying to find a mate during mating season\n\n- It was fleeing from a predator or perceived threat\n\n- It was disoriented and crossed accidentally \n\n- It was following a herd of other kangaroos who were crossing\n\n- It wanted a change of scenery or environment \n\n- It was trying to reach a new habitat or territory\n\nThe real reason is unknown without more context, but hopefully one of those potential explanations does the joke justice! Let me know if you have any other animal jokes I can try to decipher." additional_kwargs={} example=False


## 序列的回退机制
我们也可以为序列本身创建后备序列。这里我们使用两种不同的模型来实现这一点：ChatOpenAI 和普通的 OpenAI（后者不使用聊天模型）。由于 OpenAI 并非聊天模型，您可能需要使用不同的提示词。

In [30]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [31]:
# Now lets create a chain with the normal OpenAI model
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm

In [32]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

'\n\nAnswer: The turtle crossed the road to get to the other side, and I have to say he had some impressive determination.'

## 长输入回退处理
大型语言模型（LLMs）的主要限制因素之一是其上下文窗口。通常情况下，你可以在将提示发送给LLM之前计算并跟踪其长度，但在难以/复杂的情况下，可以回退到具有更长上下文长度的模型。

In [34]:
short_llm = ChatOpenAI()
long_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm = short_llm.with_fallbacks([long_llm])

In [38]:
inputs = "What is the next number: " + ", ".join(["one", "two"] * 3000)

In [40]:
try:
    print(short_llm.invoke(inputs))
except Exception as e:
    print(e)

This model's maximum context length is 4097 tokens. However, your messages resulted in 12012 tokens. Please reduce the length of the messages.


In [41]:
try:
    print(llm.invoke(inputs))
except Exception as e:
    print(e)

content='The next number in the sequence is two.' additional_kwargs={} example=False


## 回退至更优模型
我们经常要求模型以特定格式（如JSON）输出结果。像GPT-3.5这样的模型能够较好地完成这一任务，但有时也会遇到困难。这自然引出了备选方案——我们可以先尝试使用GPT-3.5（速度更快、成本更低），如果解析失败，再改用GPT-4。

In [42]:
from langchain.output_parsers import DatetimeOutputParser

In [67]:
prompt = ChatPromptTemplate.from_template(
    "what time was {event} (in %Y-%m-%dT%H:%M:%S.%fZ format - only return this value)"
)

In [75]:
# In this case we are going to do the fallbacks on the LLM + output parser level
# Because the error will get raised in the OutputParser
openai_35 = ChatOpenAI() | DatetimeOutputParser()
openai_4 = ChatOpenAI(model="gpt-4") | DatetimeOutputParser()

In [77]:
only_35 = prompt | openai_35
fallback_4 = prompt | openai_35.with_fallbacks([openai_4])

In [80]:
try:
    print(only_35.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: The Super Bowl in 1994 took place on January 30th at 3:30 PM local time. Converting this to the specified format (%Y-%m-%dT%H:%M:%S.%fZ) results in: 1994-01-30T15:30:00.000Z


In [81]:
try:
    print(fallback_4.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

1994-01-30 15:30:00
